![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# TFM - Impacto de la alimentación sobre la salud de las personas

## Table of Contents

* [A. Introduction](#introduction)
* [B. Importing Libraries](#libraries)
* [C. Importing data](#data)

## A. Introduction <a class="anchor" id="introduction"></a>

## B. Importing libraries<a class="anchor" id="libraries"></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # Series and DataFrames

# import libraries for plotting
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import preprocessing

## C. Importing data<a class="anchor" id="data"></a>

In [3]:
df_andalucia = pd.read_excel('./data/Andalucia.xlsx', index_col=0).reset_index().rename(columns={'index': 'ALIMENTO'})

df_andalucia.head()

,ALIMENTO,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,NaN,58480.38,57141.87,57290.85,59618.23,...,188132.24,192093.81,189974.88,178804.61,172582.70,175969.14,182950.03,168597.49,173629.17,172746.30
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,NaN,33395.23,32434.20,32454.36,34180.60,...,99823.54,100268.31,98722.82,92727.81,90061.88,92561.15,96130.52,89210.10,92298.34,92697.70
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,NaN,15382.91,15192.06,15131.80,16137.47,...,42888.22,44509.88,43885.76,43117.00,42111.14,43215.28,45113.61,42072.22,45120.71,46619.83
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,NaN,2126.83,1998.50,1930.66,2105.13,...,3738.19,4285.95,4105.34,3929.12,3699.01,3972.54,4068.29,3644.77,3581.93,3753.63
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NaN,397.01,389.52,375.48,404.93,...,621.83,674.72,664.54,631.64,582.11,623.85,648.24,590.65,613.55,631.47


In [4]:
df_aragon = pd.read_excel('./data/Aragon.xlsx', index_col=0).reset_index().rename(columns={'index': 'ALIMENTO'})

df_aragon.head()

,ALIMENTO,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,NaN,10894.00,9770.08,10033.98,10569.76,...,27408.29,28374.26,28781.77,27293.12,26823.82,27198.20,28762.09,25231.16,27756.19,26811.75
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,NaN,6431.27,5814.62,5956.39,6273.05,...,15189.95,15580.89,15721.40,14802.69,14694.85,14986.50,15780.06,14042.34,15282.67,14942.22
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,NaN,2916.89,2664.88,2883.38,3050.48,...,6507.77,6700.31,7029.27,6911.51,6739.31,6887.39,7324.76,6432.17,7542.61,7549.93
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,NaN,364.95,308.06,326.65,351.22,...,616.73,665.12,704.29,645.44,608.71,654.72,720.15,599.48,690.66,657.39
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NaN,76.31,72.20,77.00,73.52,...,129.64,136.09,146.14,132.20,125.64,130.72,142.37,113.97,130.95,133.01


In [5]:
df_asturias = pd.read_excel('./data/Asturias.xlsx', index_col=0).reset_index().rename(columns={'index': 'ALIMENTO'})

df_asturias.head()

,ALIMENTO,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,23628.69,23063.50,23262.42,20898.45,21060.24,22446.36,22481.75,21114.67,21963.94,21662.69
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,NaN,0,0,0,0,...,12694.37,12305.11,12368.03,11145.87,11259.31,12019.90,11968.82,11327.61,11939.20,11879.06
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,NaN,0,0,0,0,...,6037.38,6012.17,6152.34,5658.37,5622.31,6115.87,6134.09,5846.77,6361.38,6422.57
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,NaN,0,0,0,0,...,546.07,574.55,596.34,531.98,529.49,574.31,560.52,512.46,545.67,557.61
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NaN,0,0,0,0,...,66.68,72.42,75.29,66.62,63.62,71.72,73.59,67.22,67.14,71.71


In [6]:
df_baleares = pd.read_excel('./data/Baleares.xlsx', index_col=0).reset_index().rename(columns={'index': 'ALIMENTO'})

df_baleares.head()

,ALIMENTO,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,NaN,14727.87,14442.84,13911.56,13613.85,...,36886.74,36003.99,36008.64,33569.47,33103.06,32996.60,34023.32,30865.15,31354.86,29921.51
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,NaN,8759.78,8606.66,8310.17,8102.61,...,21211.59,20511.80,20482.43,19133.04,18898.09,18815.58,19292.81,17713.65,18107.42,17301.22
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,NaN,3992.89,3996.22,3928.73,3894.70,...,8708.15,8699.04,8833.35,8569.77,8204.13,8236.42,8745.11,8197.27,8629.63,8555.86
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,NaN,434.26,401.29,388.52,395.03,...,597.55,625.55,639.52,577.08,548.43,573.45,627.76,564.80,577.49,561.19
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NaN,88.03,89.28,91.27,90.66,...,97.59,98.11,104.81,99.55,95.64,98.10,110.14,99.41,105.03,104.61


In [8]:
df_c_valenciana = pd.read_excel('./data/C_valenciana.xlsx', index_col=0).reset_index().rename(columns={'index': 'ALIMENTO'})

df_c_valenciana.head()

,ALIMENTO,SECTOR,SECCION,CATEGORIA,FAMILIA,SEGMENTO BIO,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,NaN,47716.06,46107.44,45983.02,47920.00,...,133736.80,129779.19,128681.34,123079.15,118807.41,121632.29,122601.75,111520.51,119289.98,116625.88
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,NaN,27466.29,26647.20,26522.97,27729.42,...,73843.91,70412.51,69115.57,66673.94,64414.00,66138.66,66473.96,61197.84,65186.28,64374.34
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,NaN,12985.07,12786.42,12974.26,13583.18,...,31245.53,30908.99,30888.04,31025.79,29522.89,29830.58,30499.43,28506.47,31409.23,31985.94
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,NaN,1403.53,1350.94,1345.46,1445.03,...,2029.53,2113.52,2084.87,1948.98,1939.10,1975.39,1998.17,1794.29,1957.18,2001.37
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,NaN,270.72,274.69,272.85,296.13,...,361.74,382.19,375.47,380.59,361.02,363.46,378.93,349.57,371.36,378.92
